# iTorch / Torch7 implementation


**Gm = d** in lua/torch for LuaJIT. 


## Prerequisites

Works for me on Ubuntu 16.04 and MacOS 10.11. Not gonna say thte Mac installation is easy though.

- [Getting started with Torch](http://torch.ch/docs/getting-started.html)
- [iTorch Requirements](https://github.com/facebook/iTorch#requirements)
- [Torch cheatsheet](https://github.com/torch/torch7/wiki/Cheatsheet)


## Demo

We'll start with the example from [the torch7 documentation](http://torch7.readthedocs.io/en/rtd/maths/#eigenvalues-svd-linear-system-solution). 

In [1]:
G = torch.Tensor({{6.80, -2.11,  5.66,  5.97,  8.23},
                  {-6.05, -3.30,  5.36, -4.44,  1.08},
                  {-0.45,  2.58, -2.70,  0.27,  9.04},
                  {8.32,  2.71,  4.35,  -7.17,  2.14},
                  {-9.67, -5.14, -7.26,  6.08, -6.87}}):t()  -- transpose

In [2]:
d = torch.Tensor({{4.02,  6.19, -8.22, -7.57, -3.03},
                  {-1.56,  4.00, -8.67,  1.75,  2.86},
                  {9.81, -4.09, -4.57, -8.61,  8.99}}):t()

Later we can use lower-level linear algebraic functions, but for now here's the solution using LAPACK's `gesv`, which [is described](http://www.crest.iu.edu/research/mtl/reference/html/gesvLapackMatAVectorIntLapackMatB.html) like this:

> Computes the solution to a real system of linear equations `A*X=B` (simple driver). LU decomposition with partial pivoting and row interchanges is used to solve the system.

In [3]:
m_est = torch.gesv(d, G)

In [6]:
d:dist(G*m_est)

1.1808097433449e-14	


## Mauricio's example

In [7]:
-- Define model
M = 50
m = torch.zeros(M, 1)
m[{{11,15}, {}}] = 1.0
m[{{16,27}, {}}] = -0.3
m[{{28,35}, {}}] = 2.1

In [8]:
Plot = require 'itorch.Plot'

In [9]:
x = torch.range(1,m:size(1))
plot = Plot():line(x, m[{{},1}]):circle(x, m[{{},1}]):title('Model m'):draw()

In [10]:
-- Discrete kernel G
N = 20
L = 100
alpha = 0.8

In [11]:
M = 50
x = torch.range(0, M, 1) * L/(M-1)
dx = L/(M-1)
r = torch.range(0, N, 1) * L/(N-1)

In [12]:
G = torch.zeros(N, M)

In [13]:
for j = 1,M do
    for k = 1,N do
        t = torch.abs(r[{k}] - x[{j}])
        G[{{k},{j}}] = dx * torch.exp(-alpha * torch.pow(t, 2))
    end
end

In [14]:
d = G * m

In [15]:
Plot = require 'itorch.Plot'
x = torch.range(1,d:size(1))
plot = Plot():line(x, d[{{},1}]):circle(x, d[{{},1}]):title('Data d'):draw()

## Noise free: minimum norm

In [16]:
-- Compute data
d = G* m

In [17]:
-- Minimum norm solution
m_est = G:t() * torch.inverse(G * G:t()) * d
d_pred = G * m_est

In [18]:
-- Plot as before
Plot = require 'itorch.Plot'
x = torch.range(1,d_pred:size(1))
plot = Plot():line(x, d_pred[{{},1}]):circle(x, d_pred[{{},1}]):title('Predicted data d_pred'):draw()

## Plotting demo

From [the iTorch readme](https://github.com/facebook/iTorch/blob/master/README.md).

In [19]:
x1 = torch.randn(40):mul(100)
y1 = torch.randn(40):mul(100)
x2 = torch.randn(40):mul(100)
y2 = torch.randn(40):mul(100)
x3 = torch.randn(40):mul(200)
y3 = torch.randn(40):mul(200)


-- scatter plots
plot = Plot():circle(x1, y1, 'red', 'hi'):circle(x2, y2, 'blue', 'bye'):draw()
plot:circle(x3,y3,'green', 'yolo'):redraw()
plot:title('Scatter Plot Demo'):redraw()
plot:xaxis('length'):yaxis('width'):redraw()
plot:legend(true)
plot:redraw()

## BLAS and CUDA examples

From [Nicholas Leonard's examples](https://github.com/nicholas-leonard/slides/blob/master/torch7.md).

In [38]:
batchSize, inputSize, outputSize = 4, 2, 3
input = torch.FloatTensor(batchSize, inputSize):uniform(0,1)
weight = torch.FloatTensor(outputSize, inputSize):uniform(0,1)
output = torch.FloatTensor(4, 3)  -- Must give size here, AFAICT

In [41]:
output:addmm(0, output, 1, input, weight:t())  -- transpose of weight

In [42]:
output

 1.0797  0.7530  0.6347
 0.5466  0.4279  0.3232
 0.9232  0.4706  0.5357
 0.3325  0.1418  0.1918
[torch.FloatTensor of size 4x3]



How about `CudaTensor`s?

In [43]:
require 'cutorch'
input = torch.CudaTensor(batchSize, inputSize):uniform(0,1)
weight = torch.CudaTensor(outputSize, inputSize):uniform(0,1)
output = torch.CudaTensor(4,3)

output:addmm(0, output, 1, input, weight:t())

[string "require 'cutorch'..."]:1: loop or previous error loading module 'cutorch'
stack traceback:
	[C]: in function 'require'
	[string "require 'cutorch'..."]:1: in main chunk
	[C]: in function 'xpcall'
	/Users/matt/torch/install/share/lua/5.1/itorch/main.lua:209: in function </Users/matt/torch/install/share/lua/5.1/itorch/main.lua:173>
	/Users/matt/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	/Users/matt/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	/Users/matt/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	/Users/matt/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	/Users/matt/torch/install/share/lua/5.1/itorch/main.lua:381: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x010e756d10: 

In [ ]:
output

Weird, that worked before... updating `cutorch`.